# TP 2 : récursivité et conditions

Dans tout le TP, il est interdit d'utiliser des boucles (pas de `for` ni `while`).

## Petites questions

1. Définir une fonction `divise` telle que `divise a b` renvoie `true` si  `a` divise `b`, `false` sinon. Il est interdit d'utiliser `if`.
1. Définir une fonction récursive `somme` telle que `somme n` renvoie $\sum_{k=1}^n k^2$.
2. Définir une fonction récursive `u` telle que `u n` renvoie $u_n$ définie par :
$$u_0 = 42$$
$$u_{n} = 3\sqrt{u_{n - 1}} + 2}$$
3. Soient $a$ et $b$ deux entiers et $q, r$ le quotient et reste de la division euclidienne de $a$ par $b$ ($a = bq + r$). En utilisant le fait que $PGCD(a, b)$ = $PGCD(b, r)$, écrire une fonction `euclide` telle que `euclide a b` renvoie le PGCD de $a$ et $b$.

In [2]:
let divise a b = (b mod a = 0)

val divise : int -> int -> bool = <fun>


In [5]:
divise 6 2

- : bool = false


In [7]:
let rec somme n =
    if n = 1 then 1
    else somme (n-1) + n*n

val somme : int -> int = <fun>


In [9]:
somme 4

- : int = 30


In [10]:
let rec u n = 
    if n = 0. then 42.
    else 3. *. (u (n-.1.)) ** 0.5 +. 2.

val u : float -> float = <fun>


In [12]:
u 3.

- : float = 13.9593276159321622


In [13]:
let rec euclide a b = 
    if (a mod b = 0) then b
    else euclide b (a mod b)

val euclide : int -> int -> int = <fun>


In [16]:
euclide 12 18

- : int = 6


## Exponentiation rapide

1. Écrire une fonction récursive `puissance` naïve (c'est à dire très simple) telle que `puissance a n` renvoie $a^n$. Combien effectue t-elle de multiplications (en fonction de n) ?
2. Écrire une fonction récursive `exp_rapide` pour calculer $a^n$, en utilisant les relations suivantes :
$$ 
\begin{cases} 
a^n = (a^{\frac{n}{2}})^2 ~~~~~~~~~\text{si }n\text{ est pair}\\
a^{n} = a \times (a^{\frac{n-1}{2}})^2 ~~~~~\text{sinon}
\end{cases}
$$
**Remarque** : On montrera plus tard que cette 2ème version demande environ $\ln(n)$ multiplications seulement.

In [18]:
let rec puissance a n = 
    if n = 0 then 1
    else (puissance a (n-1)) * a

val puissance : int -> int -> int = <fun>


In [45]:
puissance 2 61

- : int = 2305843009213693952


In [48]:
let rec exp_rapide a n =
    if n = 0 then 1
    else if (n mod 2 = 0) then (puissance a (n/2)) * (puissance a (n/2))
    else a * (puissance a ((n-1)/2)) * (puissance a ((n-1)/2))

val exp_rapide : int -> int -> int = <fun>


In [49]:
exp_rapide 2 61

- : int = 2305843009213693952


## Temps de vol de la suite de Syracuse

La suite de Syracuse d'un entier $a$ est définie par :  
$$u_0 = a$$
$$u_{n+1} =
\begin{cases} 
     \frac{u_n}{2}, \text{si } n \text{ est pair}\\
    3u_n + 1, \text{sinon}\\
\end{cases}$$

Le temps de vol de $(u_n)_n$ est le plus petit entier $t$ tel que $u_t = 1$.  

1. Écrire une fonction récursive `temps_vol` telle que `temps_vol a` renvoie le temps de vol de $(u_n)_n$ (où $u_0 = a$).

In [16]:
let rec temps_voln a n =
    if a = 1 then n
    else if a mod 2 = 0 then temps_voln (a / 2) (n+1)
    else temps_voln (3 * a + 1) (n+1) in
let temps_vol a = temps_voln a 0 in
temps_vol 7

- : int = 16


## Fonction mystérieuse

Tester la fonction suivante, conjecturer un théorème et prouvez-le par récurrence.

In [21]:
let rec f n =
    if n > 100 then n - 10 
    else f (f (n + 11)) in
f 102

- : int = 92


interrupt: intterupt

conjecture : pour tout n <= 101, f(n) = 91  
soit pour tout n <= 101 P(n) : f(n) = 91  
Init : n = 101  
f(101) = 91  
donc P(101) est vraie  
Hérédité : Soit n <= 101  
On suppose que pour tout k appartenant à $[n,101]$, P(k) est vraie  
Montrons que P(n-1) est vraie  
f(n-1) =  
- n - 11 si n-1 > 100  
- f(f(n + 10)) sinon  
Or pour tout n <= 101, n-1 <= 100  
Donc f(n-1) = f(f(n+10))